# Custom Chatbot Project

In this project of **Course 3**, we leveraged a dataset comprising 137 key facts about the Premier League from the 2023/24 season, sourced from theanalyst.com. This dataset was selected to support the development of a custom chatbot tool designed to accuratly answers to questions specifically related to the Premier League and the 2023/24 season. To enhance the tool's question-answering capabilities, we implemented the Retrieval Augmented Generation (RAG) technique. This approach enriches the model's responses by augmenting the input prompt with contextual data from the dataset, ensuring more precise and contextually relevant answers to user queries.

In [2]:
# Environment variables
VOC_OPENAI_API_KEY = 'voc-9001790381266773650678673243c3eb54d3.20882087'

# URLs and file paths
DATA_SOURCE_URL = 'https://theanalyst.com/eu/2024/05/premier-league-best-facts-2023-24'
HTML_SOURCE_CODE = '.html_source_code.html'
EMBEDDINGS_PATH = './premier_league_embeddings.csv'

# OpenAI Models
EMBEDDING_MODEL = 'text-embedding-3-small'
CHAT_BOT_MODEL = 'gpt-3.5-turbo'

# Batch size for processing
BATCH_SIZE = 30

## Data Wrangling

In [3]:
# Importing libraries
from scipy.spatial.distance import cosine
import requests, pandas as pd
import pandas as pd
from openai import OpenAI
from bs4 import BeautifulSoup
from typing import List, Union, Dict

In [5]:
# Helper function to fetch HTML page from a URL
def scrap_html_page(url: str) -> bytes:
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.content
    else:
        raise Exception('Connection error')

# Save the HTML page to a file
with open(HTML_SOURCE_CODE, mode='wb') as html_file:
    html_page = scrap_html_page(DATA_SOURCE_URL)
    html_file.write(html_page)

### Fetch content of a HTML page using BeautifulSoup library

In [6]:
# Function to extract key facts from the HTML file about the Premier League
def get_data_from_html(html_file_path: str) -> pd.DataFrame:
    """
    Extracts relevant data from the HTML content of the given file and returns it as a DataFrame.

    Args:
        html_file_path (str): The path to the HTML file to parse and extract data from.

    Returns:
        pd.DataFrame: A DataFrame containing the extracted data, where each row represents
                      a fact about the Premier League.
    """
    with open(html_file_path, 'r') as file:
        soup = BeautifulSoup(file, 'html.parser')  # Parse the HTML content using BeautifulSoup

    # Find the first header containing the root content node
    root_content_node = soup.find('h2', {'class': 'wp-block-heading has-text-align-center'})

    # Extract the names of all months from the header elements (h2 with specific class)
    month_titles = [header.find_next('strong').text for header in soup.find_all('h2', {'class': 'wp-block-heading has-text-align-center'})]

    # Initialize variable to track the current month while processing the content
    current_month = None
    extracted_data = []  # List to store the extracted data in the desired format

    # Iterate through the DOM nodes starting from the root content node
    for html_node in root_content_node.find_all_next():
        # Check if the current node represents a new month header
        if html_node.name == 'h2' and html_node.find_next('strong') and html_node.find_next('strong').text in month_titles:
            current_month = html_node.find_next('strong').text  # Update the current month

        # If the current node is an unordered list (ul), extract its list items (li)
        elif html_node.name == 'ul' and current_month:
            # Loop through each list item within the unordered list
            for list_item in html_node.find_all('li'):
                # Append the formatted text containing the month and fact
                extracted_data.append(f"{current_month} 2024 -- {list_item.text.strip()}")

    # Create a DataFrame from the extracted list, with a column named 'text' for the facts
    df_extracted_facts = pd.DataFrame(extracted_data, columns=['text'])

    return df_extracted_facts


In [7]:
# Extract data from the HTML file and store it in a DataFrame
df_extracted_facts = get_data_from_html(HTML_SOURCE_CODE)

df_extracted_facts.head()

,text
0,September 2024 -- Jarrod Bowen became the firs...
1,"September 2024 -- With Burnley, Luton, Everton..."
2,September 2024 -- Son Heung-min’s treble again...
3,September 2024 -- Courtesy of a hat-trick agai...
4,"September 2024 -- Against Newcastle, Evan Ferg..."


### Create Embedding Database

In [4]:
# Initialize OpenAI client using Vocareum openAI key
openai_client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = VOC_OPENAI_API_KEY
)

In [8]:
# Function to retrieve embeddings from the OpenAI API
def fetch_embeddings(prompt: Union[str, List[str]], embedding_model: str) -> List[List[float]]:

    response = openai_client.embeddings.create(
        input=prompt if isinstance(prompt, list) else [prompt],
        model=embedding_model
    )
    # Extract and return embeddings from the API response
    return [row.embedding for row in response.data]

# Function to generate embeddings for text data in a DataFrame
def create_embeddings(df: pd.DataFrame, embedding_model_name: str = EMBEDDING_MODEL, batch_size: int = BATCH_SIZE) -> List[List[float]]:

    embeddings_list = []  # List to store all the embeddings generated

    # Iterate over the DataFrame in batches
    for batch_start_index in range(0, len(df), batch_size):
        # Select the batch of text data for embedding
        batch_texts = df.iloc[batch_start_index:batch_start_index + batch_size]['text'].tolist()
        # Retrieve embeddings for the current batch of texts
        batch_embeddings = fetch_embeddings(batch_texts, embedding_model_name)
        # Append the retrieved embeddings to the output list
        embeddings_list.extend(batch_embeddings)

    return embeddings_list


In [9]:
# Add embeddings to DataFrame and save to CSV
df_extracted_facts['embedding'] = create_embeddings(df_extracted_facts)
df_extracted_facts.to_csv(EMBEDDINGS_PATH, sep=',', index=False)

# Display DataFrame head
df_extracted_facts.head()

,text,embedding
0,September 2024 -- Jarrod Bowen became the firs...,"[-0.051968786865472794, 0.024718431755900383, ..."
1,"September 2024 -- With Burnley, Luton, Everton...","[-0.025487609207630157, -0.005696733482182026,..."
2,September 2024 -- Son Heung-min’s treble again...,"[-0.0037523352075368166, 0.006605617236346006,..."
3,September 2024 -- Courtesy of a hat-trick agai...,"[-0.019529273733496666, -0.0046683745458722115..."
4,"September 2024 -- Against Newcastle, Evan Ferg...","[-0.013162550516426563, 0.002083773259073496, ..."


## Custom Query Completion


In [13]:
# Function to create a simple prompt for a question
def simple_prompt(question: str) -> List[Dict[str, str]]:
    """
    Creates a simple prompt for a user question without additional context.

    Args:
        question (str): The question the user is asking.

    Returns:
        List[Dict[str, str]]: A list of dictionaries representing the user message.
    """
    return [
        {
            'role': 'user',
            'content': question
        }
    ]

# Function to create a custom prompt with a context based on the question and a database of facts
def custom_prompt(question: str, database_df: pd.DataFrame) -> List[Dict[str, str]]:
    """
    Creates a custom prompt that includes a context for answering the question. The context
    is based on relevant data from the provided DataFrame (database).

    Args:
        question (str): The question the user is asking.
        database_df (pd.DataFrame): DataFrame containing context (e.g., facts) related to the question.

    Returns:
        List[Dict[str, str]]: A list of dictionaries representing the system and user messages.
    """
    return [
        {
            'role': 'system',
            'content': """
            Answer the question based on the provided context below. If the question cannot be answered based on the context, say "I don't know the answer".
            The context contains 137 key facts about the Premier League from the 2023/24 season, sourced from theanalyst.com. Facts are labeled with dates and divided by lines..
            Context:
                {}
            """.format('\n\n'.join(get_custom_context(question, database_df)))
        },
        {
            'role': 'user',
            'content': question
        }
    ]

# Function to build the context by retrieving relevant facts from the database based on the question
def get_custom_context(question: str, database_df: pd.DataFrame, n: int = 10) -> List[str]:
    """
    Builds a custom context for the question by selecting the top 'n' most relevant facts from the database.

    Args:
        question (str): The question the user is asking.
        database_df (pd.DataFrame): DataFrame containing context (facts).
        n (int): The number of most relevant facts to include in the context (default is 10).

    Returns:
        List[str]: A list of strings representing the relevant facts to be included in the context.
    """
    # Retrieve the embedding for the question
    question_embedding = fetch_embeddings(question, EMBEDDING_MODEL)[0]

    # Create a copy of the database and compute distances between the question embedding and each fact's embedding
    df_copy = database_df.copy()
    df_copy["distances"] = df_copy['embedding'].apply(lambda embedding: cosine(embedding, question_embedding))

    # Sort the DataFrame by the computed distances (similarity), and select the top 'n' relevant facts
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy.iloc[:n]['text'].tolist()

# Function to handle the question by sending the prompt to the OpenAI client and receiving a response
def answer_query(prompt: List[Dict[str, str]], client: OpenAI, model_name: str = CHAT_BOT_MODEL) -> str:
    """
    Sends a prompt to the OpenAI API and retrieves the response from the specified model.

    Args:
        prompt (List[Dict[str, str]]): The prompt containing user and system messages.
        client (OpenAI): The OpenAI client used to interact with the API.
        model_name (str): The name of the model to use for generating the response (default is `CHAT_BOT_MODEL`).

    Returns:
        str: The content of the response from the OpenAI model.
    """
    # Request a completion (response) from the model using the provided prompt
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        max_tokens=1000  # Set the maximum number of tokens for the response
    )

    # Return the content of the response message
    return response.choices[0].message.content


## Custom Performance Demonstration


In [14]:
# Read the DataFrame from CSV file
df_embeddings = pd.read_csv(EMBEDDINGS_PATH)

# Convert embedding values from string to list of floats
df_embeddings['embedding'] = df_embeddings['embedding'].apply(lambda value: [float(dim) for dim in value.replace('[', '').replace(']', '').split(',')])

### Question 1

In [15]:
# Define the question
first_query = 'Who became the became the first player to score in each of West Ham’s first four away games in a top-flight season since Vic Watson in 1929-30?'

# Print answer without context
print('Answer without Context: \n', answer_query(simple_prompt(first_query), openai_client))

# Print answer with context
print('\nAnswer with Context: \n', answer_query(custom_prompt(first_query, df_embeddings), openai_client))

Answer without Context: 
 Michail Antonio became the first player to score in each of West Ham's first four away games in a top-flight season since Vic Watson in 1929-30.

Answer with Context: 
 Jarrod Bowen became the first player to score in each of West Ham’s first four away games in a top-flight season since Vic Watson in 1929-30.


### Question 2

In [16]:
# Define the question
second_query = 'Which was the latest winning goal ever scored in a Premier League fixture between Arsenal and Manchester United.'

# Print answer without context
print('Answer without Context: \n', answer_query(simple_prompt(second_query), openai_client))

# Print answer with context
print('\nAnswer with Context: \n', answer_query(custom_prompt(second_query, df_embeddings), openai_client))

Answer without Context: 
 The latest winning goal ever scored in a Premier League fixture between Arsenal and Manchester United was netted by Danny Welbeck in the 95th minute on February 28, 2016. This goal gave Arsenal a 3-2 victory over Manchester United at Old Trafford.

Answer with Context: 
 Declan Rice’s goal on 95 minutes and 43 seconds in September 2024 was the latest winning goal ever scored in a Premier League fixture between Arsenal and Manchester United.
